In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import glob, os

In [ ]:
def read_data_shield(fn, reindex=False):
    data = pd.read_csv(fn, index_col=0, parse_dates=True)
    if reindex:
        data.index = pd.DatetimeIndex(start=data.index[0],
                                      end=data.index[-1],
                                      periods=len(data)
                                     )
    return data

folder = r'd:\git\elevation-barometry\data\closed_room'
fns = glob.glob(os.path.join(folder, '*.csv'))
fns.sort()

raw_data = [read_data_shield(fn, reindex=True) for fn in fns]
data_start = raw_data[0].index[0]
index_re = raw_data[0].index
data_re = []
var = raw_data[0].keys()[-1]
for data in raw_data:
    diff = data.index[0]-data_start
    data.index = data.index-diff
    data_re.append(data[var].reindex(index_re, method='nearest', tolerance=pd.Timedelta(100, 'ms')))

ax = plt.subplot(111)
pressure = pd.concat(data_re, axis=1, keys=['sensor 1 [Pa]', 'sensor 2 [Pa]', 'sensor 3 [Pa]', 'sensor 4 [Pa]'])
pressure[1000:].plot(ax=ax, legend=False)
ax.set_ylabel('Pressure [Pa]')
# plt.savefig('Pressure_ts.png', dpi=300, bbox_inches='tight')
# data1 = raw_data[0][var][1000:].values
# data2 = raw_data[1][var][1000:].values
# print(len(data1), len(data2))
# diff = data1[:len(data2)]-data2

# diff.std()

# data = read_data_shield(fn, reindex=True)
# data[var][1000:].plot()
# plt.plot(data[var][0:100].values) #.plot()

In [ ]:
temp = raw_data[0][raw_data[0].keys()[0]]
pres = raw_data[0][raw_data[0].keys()[1]]
plt.plot(temp, pres, '.')

In [ ]:
# EXPERIMENT!
# show constantness of bias
# pd.plotting.scatter_matrix(pressure, alpha = 0.3, figsize = (14,8), diagonal = 'kde')
# plt.savefig('oven_scatter.png', dpi=300, bbox_inches='tight')



# show autocorrelation diagram


In [ ]:
# perform cross-over variance test, comparison with theoretical error
import numpy as np
from  itertools import combinations
start = 1000
end = 46000
wins = np.arange(1, 500)

cc = list(combinations(pressure.columns, 2))
labels = ['sensor {:s}-{:s}'.format(key1.strip(' [Pa]').strip('sensor '), key2.strip(' [Pa]').strip('sensor ')) for key1, key2 in cc]
labels
cc

std = [[(pressure[key1][-end:]-pressure[key2][-end:]).rolling(window=win).mean().std() for win in wins] for key1, key2 in cc]

# .rename(columns=lambda x: x + 1)
# std = _.transpose().rename(lambda x: x + 1)

# also make an idealized white noise reduction with central limit assumption
white_noise_1 = [std[n][0:1]/np.sqrt(wins) for n in range(len(std))]

# glue together
# axs = []

plt.figure(figsize=(9, 4))
ax = plt.subplot(1, 1, 1)
for n, (std_, white_noise, label) in enumerate(zip(std, white_noise_1, labels)):
#     ax = plt.subplot(len(std)/2, 2, n + 1)
#     axs.append(ax)
    if n == 0:
        label = 'sensor error'
    else:
        label=None
    l1 = ax.plot(np.arange(len(std_)), std_, color='grey', marker='.', linewidth=0., label=label)
l2 = ax.plot(np.arange(len(std_)), white_noise, label='theoretical error')
#     ax.add_xlabel('Window size')
#     ax.add_ylabel('$\sigma$ [Pa]')
ax.legend()
plt.xlabel('window size [0.1 sec]')
plt.ylabel('$\sigma$ [Pa]')
plt.savefig('error.png', dpi=300, bbox_inches='tight')
# plt.title('Impact of averaging on variance')
# ax = plt.subplot(212)
# ax.plot(np.arange(len(std[1])), std[1], marker='.', label='estimated mean')
# ax.plot(np.arange(len(std[1])), white_noise_1[1], label='central limit mean')
# plt.xlabel('Window size')
# plt.ylabel('$\mu$ [Pa]')


In [ ]:
start=2000
bar_const = 8485.921788

diffs = [(pressure[key1][start:]-pressure[key2][start:]).rolling(window=1500).mean() -(pressure[key1][start:end]-pressure[key2][start:end]).mean() for key1, key2 in cc] # 
diffs_z = []
for key1, key2 in cc:
    pressure1 = pressure[key1]
    pressure2 = pressure[key2]
    bias = (pressure1[start:end] - pressure2[start:end]).mean()
    # add bias to pressure 2
    pressure2 += bias
    # now compute dz
    dz = (100*bar_const*np.log(pressure1[start:]/pressure2[start:])).rolling(window=1500).mean()
    diffs_z.append(dz)

f = plt.figure(figsize=(12, 3))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

for diff, diff_z in zip(diffs, diffs_z):
    diff.plot(ax=ax1, alpha=0.3, color='k')
    diff_z.plot(ax=ax2, alpha=0.3, color='k')
ax1.set_ylabel('Pressure difference [Pa]')
ax2.set_ylabel('Elevation difference [cm]')
ax1.set_xlabel('time [DD-MM HH]')
ax2.set_xlabel('time [DD-MM HH]')
plt.savefig('Drift.png', dpi=300, bbox_inches='tight')


In [ ]:
len(diffs_z)